# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 16 2023 8:30AM,259401,10,U1240,"Uniderm USA, Inc",Executing
1,Mar 16 2023 8:30AM,259401,10,U1239,"Uniderm USA, Inc",Released
2,Mar 16 2023 8:30AM,259402,10,SO94567,"Senseonics, Incorporated",Released
3,Mar 16 2023 8:30AM,259404,10,Yusen-9270123,Yusen – 3D Matrix,Released
4,Mar 16 2023 8:30AM,259396,10,LUPLUP0021,"Lupin Research, Inc.",Released
5,Mar 15 2023 4:05PM,259403,16,Yusen-9270122,Yusen – 3D Matrix,Released
6,Mar 15 2023 3:44PM,259400,16,SHL-9270118,"SHL Pharma, LLC",Released
7,Mar 15 2023 3:38PM,259399,21,849474,"NBTY Global, Inc.",Executing
8,Mar 15 2023 3:26PM,259398,15,PNC435473A,"Person & Covey, Inc.",Executing
9,Mar 15 2023 3:26PM,259398,15,PNC435474A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,259401,Executing,1
13,259401,Released,1
14,259402,Released,1
15,259403,Released,1
16,259404,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
259400,NaN,1.0
259401,1.0,1.0
259402,NaN,1.0
259403,NaN,1.0
259404,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
259219,13.0,5.0
259319,1.0,0.0
259379,0.0,1.0
259387,1.0,1.0
259396,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
259219,13,5
259319,1,0
259379,0,1
259387,1,1
259396,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,259219,13,5
1,259319,1,0
2,259379,0,1
3,259387,1,1
4,259396,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,259219,13,5
1,259319,1,
2,259379,,1
3,259387,1,1
4,259396,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 16 2023 8:30AM,259401,10,"Uniderm USA, Inc"
2,Mar 16 2023 8:30AM,259402,10,"Senseonics, Incorporated"
3,Mar 16 2023 8:30AM,259404,10,Yusen – 3D Matrix
4,Mar 16 2023 8:30AM,259396,10,"Lupin Research, Inc."
5,Mar 15 2023 4:05PM,259403,16,Yusen – 3D Matrix
6,Mar 15 2023 3:44PM,259400,16,"SHL Pharma, LLC"
7,Mar 15 2023 3:38PM,259399,21,"NBTY Global, Inc."
8,Mar 15 2023 3:26PM,259398,15,"Person & Covey, Inc."
18,Mar 15 2023 3:25PM,259219,15,"Person & Covey, Inc."
36,Mar 15 2023 3:25PM,259387,15,"Virtus Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 16 2023 8:30AM,259401,10,"Uniderm USA, Inc",1,1
1,Mar 16 2023 8:30AM,259402,10,"Senseonics, Incorporated",,1
2,Mar 16 2023 8:30AM,259404,10,Yusen – 3D Matrix,,1
3,Mar 16 2023 8:30AM,259396,10,"Lupin Research, Inc.",,1
4,Mar 15 2023 4:05PM,259403,16,Yusen – 3D Matrix,,1
5,Mar 15 2023 3:44PM,259400,16,"SHL Pharma, LLC",,1
6,Mar 15 2023 3:38PM,259399,21,"NBTY Global, Inc.",1,
7,Mar 15 2023 3:26PM,259398,15,"Person & Covey, Inc.",8,2
8,Mar 15 2023 3:25PM,259219,15,"Person & Covey, Inc.",13,5
9,Mar 15 2023 3:25PM,259387,15,"Virtus Pharmaceuticals, LLC",1,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 8:30AM,259401,10,"Uniderm USA, Inc",1,1
1,Mar 16 2023 8:30AM,259402,10,"Senseonics, Incorporated",1,
2,Mar 16 2023 8:30AM,259404,10,Yusen – 3D Matrix,1,
3,Mar 16 2023 8:30AM,259396,10,"Lupin Research, Inc.",1,
4,Mar 15 2023 4:05PM,259403,16,Yusen – 3D Matrix,1,
5,Mar 15 2023 3:44PM,259400,16,"SHL Pharma, LLC",1,
6,Mar 15 2023 3:38PM,259399,21,"NBTY Global, Inc.",,1
7,Mar 15 2023 3:26PM,259398,15,"Person & Covey, Inc.",2,8
8,Mar 15 2023 3:25PM,259219,15,"Person & Covey, Inc.",5,13
9,Mar 15 2023 3:25PM,259387,15,"Virtus Pharmaceuticals, LLC",1,1


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 8:30AM,259401,10,"Uniderm USA, Inc",1,1
1,Mar 16 2023 8:30AM,259402,10,"Senseonics, Incorporated",1,
2,Mar 16 2023 8:30AM,259404,10,Yusen – 3D Matrix,1,
3,Mar 16 2023 8:30AM,259396,10,"Lupin Research, Inc.",1,
4,Mar 15 2023 4:05PM,259403,16,Yusen – 3D Matrix,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 8:30AM,259401,10,"Uniderm USA, Inc",1.0,1.0
1,Mar 16 2023 8:30AM,259402,10,"Senseonics, Incorporated",1.0,NaN
2,Mar 16 2023 8:30AM,259404,10,Yusen – 3D Matrix,1.0,NaN
3,Mar 16 2023 8:30AM,259396,10,"Lupin Research, Inc.",1.0,NaN
4,Mar 15 2023 4:05PM,259403,16,Yusen – 3D Matrix,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 8:30AM,259401,10,"Uniderm USA, Inc",1.0,1.0
1,Mar 16 2023 8:30AM,259402,10,"Senseonics, Incorporated",1.0,0.0
2,Mar 16 2023 8:30AM,259404,10,Yusen – 3D Matrix,1.0,0.0
3,Mar 16 2023 8:30AM,259396,10,"Lupin Research, Inc.",1.0,0.0
4,Mar 15 2023 4:05PM,259403,16,Yusen – 3D Matrix,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1296922,4.0,2.0
15,778004,8.0,22.0
16,518803,2.0,0.0
18,259379,1.0,0.0
19,259397,1.0,0.0
21,259399,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1296922,4.0,2.0
1,15,778004,8.0,22.0
2,16,518803,2.0,0.0
3,18,259379,1.0,0.0
4,19,259397,1.0,0.0
5,21,259399,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,4.0,2.0
1,15,8.0,22.0
2,16,2.0,0.0
3,18,1.0,0.0
4,19,1.0,0.0
5,21,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,4.0
1,15,Released,8.0
2,16,Released,2.0
3,18,Released,1.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,21
Status,,,,,,
Executing,2.0,22.0,0.0,0.0,0.0,1.0
Released,4.0,8.0,2.0,1.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,21
0,Executing,2.0,22.0,0.0,0.0,0.0,1.0
1,Released,4.0,8.0,2.0,1.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,21
0,Executing,2.0,22.0,0.0,0.0,0.0,1.0
1,Released,4.0,8.0,2.0,1.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()